In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt

In [ ]:
def linear_costs(x, y, z, theta):
    costs = (np.matmul(x,theta)-z)*(1-2*y)
    return costs

In [ ]:
def paired_classifier(x, costs):
    f = []
    for cost in costs.T:
        f.append(LinearRegression(fit_intercept=False).fit(x, cost))
    return f

In [ ]:
def obj_grad(objective, x, z, theta, n):
    zhat = np.matmul(x,theta)
    zdiff = zhat - z
    
    if objective=='MSE':
        grad = 2*np.matmul(x.T,zdiff)/n
        #Do we really want "flatten" here? This goes from a multi- to 1-D array
        return grad.flatten()
    
    elif objective=='dot_product':
        grad = np.matmul(x.T,z)/n
        return grad.flatten()
    
    else:
        return 0

In [ ]:
def has_converged(x, size, epsilon, d):
    for i in range(0,size):
        for j in range(0,d):
            if np.linalg.norm(x[-1:][j] - x[-(2+i):][j], np.inf) > epsilon:
                return False
    return True

In [ ]:
def linear_proxies(x, y, z_all, C, iters, n, K, d, epsilon, theta_true, objective=None):
    theta_list = []
    grad_list = []
    theta_average_list = []
    
    for k in range(0,K):
        #Be very careful about dimensionality here -- again, reduce to simplest case
        z = z_all[:,k].reshape(-1,1)

        theta = [np.random.rand(d,1)]
        grad_l = [0]
        theta_average = [0]
        converged = False
        
        for t in range(1, iters):
            if converged:
                break
                
            costs = linear_costs(x, y, z, theta[t-1])
            f_s = paired_classifier(x, costs)
            
            ###
            #Something is weird here with dimensions of h
            #Separate this for multiple y case and write version for one y -- good sanity check
            
            h = [0]*n
            val = []
            
            for i, f in enumerate(f_s):
                h_s = f.predict(x)
                h_plus = (h_s > 0)
                h_minus = (h_s < 0)
                if np.abs(np.sum(h_s[h_plus]))>np.abs(np.sum(h_s[h_minus])):
                    h[i] = h_plus
                else:
                    h[i] = h_minus
                val.append(np.sum(h_s[h[i]]))
                
            y_index = np.argmin(val)
            y_temp = y[:,y_index]
            h = h[y_index].astype(int)
            ####
            
            zhat = np.matmul(x,theta[t-1])
            zhat_sum = np.sum(zhat)
            z_sum = np.sum(z)
            
            #TODO: Add this at end, might correct things
            err_points = np.abs(h-y_temp)
            err_cost = np.matmul((zhat-z).T,err_points)
            overall_diff = (zhat_sum/z_sum) - 1
            
            #Double check that types work out here, kinda weird with flatten
            if np.abs(overall_diff) >= np.abs(err_cost):
                penalty = np.sign(overall_diff) * np.sum(x, axis=0)/z_sum
            else:
                penalty = np.sign(err_cost) * np.matmul(np.transpose(x),err_points)  
                penalty = penalty.flatten()
            
            #Work on MSE objective
            grad_l.append(obj_grad(objective,x,z,theta[t-1],n) + C * penalty) 
            
            #Can we cut down storage space to speed up here?
            theta.append(theta[t-1] - (np.power(t, -1/2) * grad_l[t]).reshape(-1,1))
            theta_average.append((t*theta_average[t-1]+theta[t])/(t+1))
            
            if t > 10:
                if has_converged(theta_average, 10, epsilon, 1):
                    print("Converged")
                    converged = True
            
        theta_list.append(theta)
        grad_list.append(grad_l)
        theta_average_list.append(theta_average[-1])
        #TODO: sanity check that theta_average_list entry is same thing as average of thetas
        
    return theta_list, grad_list, theta_average_list

In [ ]:
def calculate_hyperparameters(d, B, z, n, M, alpha):
    z_sum = np.sum(z)
    alpha_exp = alpha*z_sum/(1+n*M)
    
    C = (M**2+2*alpha_exp)/alpha_exp
    T = d**4*np.square(2*M*B+n*C*B/z_sum)/np.square(alpha_exp)
    return C, int(T)


In [ ]:
#Make into function
#4

np.random.seed(1)

#n=10
n = 5
m = 1
K = n
d = n

M = 2
B = 1

epsilon = 0.00001
alpha=0.1
trials=5

intercept = np.ones(n).reshape(-1,1)
discrepancy = []
    
for i in range(0,trials):
    z_train = np.round(np.random.rand(n,K))
    
    #Append intercept or lack thereof here? Can we make sub structures? I think number of groups 
    #might be getting confused with something else
    
    theta = np.random.rand(n,n)
    x_train = np.matmul(z_train, np.linalg.inv(theta))
    x_train = np.hstack((intercept, x_train))
    y_train = np.round(np.random.rand(n,m))

    #Revisit this
    C, T = calculate_hyperparameters((d+1), B, z_train, n, M, alpha)
    #Fix up these parameters with new structure
    
    #z_train = z_train[:,0].reshape(-1,1)
    
    coefficients, gradients, theta_average = linear_proxies(x_train, y_train, z_train, C, T, n, 1, (d+1), epsilon, theta[:,0])
    
    z_train = z_train[:,0].reshape(-1,1)
    
    final_costs = linear_costs(x_train, y_train, z_train, theta_average)
    
    [prc] = paired_classifier(x_train, final_costs)
    
    h_s = prc.predict(x_train)
    h_plus = (h_s > 0)
    h_minus = (h_s < 0)
    
    if np.abs(np.sum(h_s[h_plus]))>np.abs(np.sum(h_s[h_minus])):
        h = h_plus
    else:
        h = h_minus

    h_int = h.astype(int)
    h_err = (h_int != y_train)
    z_hat = np.matmul(x_train, theta_average)[0]
    print(z_hat)
    
    for i in range(0,1):
        if sum(h_err[:,i])==0:
            discrepancy.append(0)
        else:
            discrepancy.append(np.mean(z_train[h_err[:,i]])/(np.mean(z_train)) - np.mean(z_hat[h_err[:,i]])/(np.mean(z_hat)))
    
print(discrepancy)
print(np.mean(discrepancy))

In [ ]:
plt.hist(discrepancy)

#Portion of discrepancies that fall in good range